In [1]:
import keras

Using TensorFlow backend.


In [2]:
import pandas as pd
from pathlib import Path
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D,Flatten,Dense,BatchNormalization,MaxPooling2D,GlobalAveragePooling2D
import numpy

#load data from local machine data directory

train_data_directory='E:/ML_projects/10-monkey-species/training/training'
validation_data_directory='E:/ML_projects/10-monkey-species/validation/validation'


In [3]:
#instead of loading all images into memory, we will use ImageDataGenerator class of keras
#That will load just enough images into memory for the current and perhaps the next few mini-batches when training and evaluating a deep learning model.
#This is progressive loading,as the dataset is progressively loaded from file,retrieving just enough data for what is needed immediately.
#we can retrieve an iterator by calling the flow_from_directory() function

train_datagen = ImageDataGenerator( 
    width_shift_range=0.1,  # randomly shift images horizontally (10% of total width)
    height_shift_range=0.1,  # randomly shift images vertically (10% of total height)
    rotation_range=40,
    zoom_range = 0.1,
    rescale=1./255)
train_generator = train_datagen.flow_from_directory(
    directory=train_data_directory,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=64,
    class_mode="categorical",#because we have 10 categories of monkey
    shuffle=True, #true means no specific order is specified
    seed=42
)
# Valid generator

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory(
    directory=validation_data_directory,
    target_size=(224, 224),
    color_mode="rgb",
    batch_size=32,
    class_mode="categorical",
    shuffle=True,
    seed=42
)

Found 1098 images belonging to 10 classes.
Found 272 images belonging to 10 classes.


In [4]:
model = Sequential()
model.add(BatchNormalization(input_shape=(224, 224, 3)))
model.add(Conv2D(filters=512, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(BatchNormalization())
model.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(BatchNormalization())
model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
model.add(MaxPooling2D(pool_size=2))

model.add(BatchNormalization())
#model.add(Conv2D(filters=128, kernel_size=3, activation='relu'))
#model.add(MaxPooling2D(pool_size=2))

#model.add(BatchNormalization())
#model.add(Conv2D(filters=256, kernel_size=3, activation='relu'))
#model.add(BatchNormalization())
#model.add(Conv2D(filters=512, kernel_size=3, activation='relu'))
#model.add(MaxPooling2D(pool_size=2))

#model.add(BatchNormalization())

#model.add(GlobalAveragePooling2D())

model.add(Flatten())
model.add(Dense(64))
model.add(Dense(32))

model.add(Dense(10, activation='softmax'))
model.summary()








Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
batch_normalization_1 (Batch (None, 224, 224, 3)       12        
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 222, 222, 512)     14336     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 111, 111, 512)     0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 111, 111, 512)     2048      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 109, 109, 256)     1179904   
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 54, 54, 256)       0         
_________________________________________________________________
batch_normalization_3 (Batch (None, 54, 54, 256

In [ ]:
#compile the model
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau

optimizer = optimizers.RMSprop(lr=0.001, rho=0.9, epsilon=None, decay=0.0)

model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=2, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=valid_generator.n//valid_generator.batch_size

model.fit_generator(generator=train_generator,
                    steps_per_epoch=STEP_SIZE_TRAIN,
                    validation_data=valid_generator,
                    validation_steps=STEP_SIZE_VALID,
                    epochs=20,
                    verbose=1,
                    callbacks=[learning_rate_reduction]
)


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Epoch 1/1
